# Conversational Interface - Chatbot with LLM

> 이 노트북은 SageMaker Studio의 **`Data Science 3.0`** 커널과 잘 작동합니다.
이 노트북에서는 Amazon Bedrock의 기본 모델 (FM) 을 사용하여 챗봇을 구축할 것입니다.

---
### 중요
- 실행시에는 **"과금"** 이 발생이 되는 부분 유념 해주시기 바랍니다.

## Setting
 - Auto Reload
 - path for utils

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os
module_path = "../../.."
sys.path.append(os.path.abspath(module_path))

## 1. Bedrock Client 생성

In [3]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

### ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----
- os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
- os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
- os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
- os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."

In [4]:
boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

print (colored("\n== FM lists ==", "green"))
pprint (bedrock_info.get_list_fm_models())

Create new client
  Using region: None
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)

== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Cohere-Embeddings-En': 'cohere.embed-english-v3',
 'Cohere-Embeddings-Multilingual': 'cohere.embed-multilingual-v3',
 'Command': 'cohere.command-text-v14',
 'Command-Light': 'cohere.command-light-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Llama2-13b-Chat': 'meta.llama2-13b-chat-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'amazon.titan-text-express-v1',
 'Titan-Text-G1-Light': 'amazon.titan-text-lite-v1'}


# 2. QA Chatbot

## 상황에 맞는 챗봇
이 사용 사례에서는 Chatbot에게 이전에 본 적이 없는 외부 코퍼스의 질문에 답변하도록 요청합니다. 이를 위해 RAG(Retrieval Augmented Generation)라는 패턴을 적용합니다. 아이디어는 말뭉치를 덩어리로 인덱싱한 다음 덩어리와 질문 사이의 의미론적 유사성을 사용하여 말뭉치의 어느 섹션이 답변을 제공하는 데 관련될 수 있는지 찾는 것입니다. 마지막으로 가장 관련성이 높은 청크가 집계되어 기록을 제공하는 것과 유사하게 ConversationChain에 컨텍스트로 전달됩니다.

**Titan Embeddings Model**을 사용하여 벡터를 생성하겠습니다. 그런 다음 이 벡터는 메모리 내 벡터 데이터 저장소를 제공하는 Amazon OpenSearch에 저장됩니다. 챗봇이 질문을 받으면 OpenSearch에 질문을 쿼리하고 의미상 가장 가까운 텍스트를 검색합니다. 이것이 우리의 대답이 될 것입니다.

### Amazon Titan Embedding 모델 사용 
- 이 모델은 최대 8,192 Token 입력이 가능합니다.

In [5]:
from langchain.embeddings import BedrockEmbeddings

In [6]:
bedrock_embeddings = BedrockEmbeddings(
    client=boto3_bedrock,
    model_id=bedrock_info.get_model_id(
        model_name="Titan-Embeddings-G1"
    )
)

### OpenSearch Client 생성
### 선수 조건
- 01_preprocess_docs/02_load_docs_opensearch.ipynb를 통해서 OpenSearch Index 가 생성이 되어 있어야 합니다.
#### [중요] 아래에 aws parameter store 에 아래 인증정보가 먼저 입력되어 있어야 합니다.
- 01_preprocess_docs/01_parameter_store_example.ipynb 참고

In [7]:
from utils.proc_docs import get_parameter

In [8]:
ssm = boto3.client("ssm", "us-east-1")

opensearch_domain_endpoint = get_parameter(
    boto3_client = ssm,
    parameter_name = 'knox_opensearch_domain_endpoint',
)

opensearch_user_id = get_parameter(
    boto3_client = ssm,
    parameter_name = 'knox_opensearch_userid',
)

opensearch_user_password = get_parameter(
    boto3_client = ssm,
    parameter_name = 'knox_opensearch_password',
)

In [9]:
opensearch_domain_endpoint = opensearch_domain_endpoint
rag_user_name = opensearch_user_id
rag_user_password = opensearch_user_password

http_auth = (rag_user_name, rag_user_password) # Master username, Master password

### Index 이름 셋팅
- 이전 노트북 01_preprocess_docs/02_load_docs_opensearch.ipynb를 통해서 생성된 OpenSearch Index name 입력

In [10]:
index_name = "genai-demo-knox-v1"

### OpenSearch Client 생성

In [11]:
from utils.opensearch import opensearch_utils

In [12]:
aws_region = os.environ.get("AWS_DEFAULT_REGION", "us-east-1")
os_client = opensearch_utils.create_aws_opensearch_client(
    aws_region,
    opensearch_domain_endpoint,
    http_auth
)

### LangChain OpenSearch VectorStore 생성

In [13]:
from langchain.vectorstores import OpenSearchVectorSearch

In [14]:
vector_db = OpenSearchVectorSearch(
    index_name=index_name,
    opensearch_url=opensearch_domain_endpoint,
    embedding_function=bedrock_embeddings,
    http_auth=http_auth, # http_auth
    is_aoss =False,
    engine="faiss",
    space_type="l2"
)
vector_db

### Hybrid search
- LangChain에서 제공하는 **BaseRetriever** 클래스를 상속받아 **Custom Retriever**를 정의 할 수 있습니다.
- 본 샘플코드 에서는 **Hybrid Search based Retriever**를 **정의**합니다. 

OpenSearch Hybrid 는 아래와 같은 방식으로 작동합니다.
- (1) Sematic serch를 통해 각 document별 relevant score 산출
- (2) Lexical search를 통해 각 document별 relevant score 산출
- (3-1) Rank-fusion 방식이 "simple weighted" 일 경우
    - 산출된 score에 대한 normalization 수행
    - 전체 결과에서 가장 높은 스코어는 표준화 과정을 통하여 스코어가 1.0 이 됨.
- (3-2) Rank-fusion 방식이 "Reciprocal Rank Fusion (RRF)" 일 경우
    - Paper: https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf
    - Desc: https://medium.com/@sowmiyajaganathan/hybrid-search-with-re-ranking-ff120c8a426d
    - **RRF의 경우 score가 아닌 ranking 정보를 활용, 때문에 score normalization이 필요 없음**
    - ![rrf.png](../../../10_advanced_question_answering/img/rrf.png)

RRF는 langchain에서 "Ensemble Retriever" 이름으로 api를 제공합니다. 
- https://python.langchain.com/docs/modules/data_connection/retrievers/ensemble

In [15]:
from utils.rag import OpenSearchHybridSearchRetriever

In [16]:
opensearch_hybrid_retriever = OpenSearchHybridSearchRetriever(
    os_client=os_client,
    vector_db=vector_db,
    index_name=index_name,

    # option for lexical
    minimum_should_match=0,
    filter=[],

    # option for rank fusion
    fusion_algorithm="RRF", # ["RRF", "simple_weighted"], rank fusion 방식 정의
    ensemble_weights=[.5, .5], # [for lexical, for semantic], Lexical, Semantic search 결과에 대한 최종 반영 비율 정의
    reranker=True, # enable reranker with reranker model
    reranker_endpoint_name="huggingface-pytorch-inference-2023-11-15-04-37-45-120", # endpoint name for reranking model

    # option for async search
    async_mode=True,
    
    # option for output
    k=5, # 최종 Document 수 정의
    verbose=False,
)

### 메모리
모든 챗봇에는 사용 사례에 따라 맞춤화된 다양한 옵션을 갖춘 QA 체인이 필요합니다. 그러나 챗봇에서는 모델이 답변을 제공하기 위해 이를 고려할 수 있도록 항상 대화 기록을 보관해야 합니다. 이 예에서는 대화 기록을 유지하기 위해 ConversationBufferMemory와 함께 LangChain의 [ConversationalRetrievalChain](https://python.langchain.com/docs/modules/chains/popular/chat_Vector_db)을 사용합니다.

출처: https://python.langchain.com/docs/modules/chains/popular/chat_Vector_db

뒤에서 무슨 일이 일어나고 있는지 모두 보려면 'verbose'를 'True'로 설정하세요.

### Conversation momory
* **[ConversationBufferMemory](https://python.langchain.com/docs/modules/memory/types/buffer)**
    - This memory allows for storing messages and then extracts the messages in a variable.
* **[ConversationBufferWindowMemory](https://python.langchain.com/docs/modules/memory/types/buffer_window)**
    - It keeps a list of the interactions of the conversation over time.
    - It only uses the last K interactions.
    - This can be useful for keeping a sliding window of the most recent interactions, so the buffer does not get too large.
* **[ConversationSummaryBufferMemory](https://python.langchain.com/docs/modules/memory/types/summary_buffer)**
    - It maintains a summary of previous messages.
    - It combines the two ideas.
    - It keeps a buffer of recent interactions in memory, but rather than just completely flushing old interactions it compiles them into a summary and uses both. 
    - It uses token length rather than number of interactions to determine when to flush interactions.

In [17]:
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT

In [18]:
print (CONDENSE_QUESTION_PROMPT.template)

Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:


### ConversationRetrievalChain에 사용되는 매개변수
* **retriever**: 우리는 `OpenSearch`를 기반으로 customization한 `OpenSearchHybridSearchRetriever`를 사용했습니다. 이것은 `OpenSearch`를 이용하여 주어진 `query`에 대한 `lexical`, `sematic` search를 기반으로 최종 후보 context 를 선택해 줍니다.

* **메모리**: 이력을 저장하는 메모리 체인

* **condense_question_prompt**: 사용자의 질문이 주어지면 이전 대화와 해당 질문을 사용하여 독립형 질문을 구성합니다.

* **chain_type**: 채팅 기록이 길고 상황에 맞지 않는 경우 이 매개변수를 사용하고 옵션은 `stuff`, `refine`, `map_reduce`, `map-rerank`입니다.

질문이 컨텍스트 범위를 벗어나면 모델은 답을 모른다고 대답합니다.

**참고**: 체인이 어떻게 작동하는지 궁금하다면 `verbose=True` 줄의 주석 처리를 해제하세요.

In [19]:
# turn verbose to true to see the full logs and documents
from langchain.schema import BaseMessage
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

from utils.chat import chat_utils
from langchain.llms.bedrock import Bedrock
from langchain.prompts import PromptTemplate
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [20]:
llm_text = Bedrock(
    model_id=bedrock_info.get_model_id(model_name="Claude-V2"),
    client=boto3_bedrock,
    model_kwargs={
        "max_tokens_to_sample": 1000,
        "temperature": 0,
        "top_k": 250,
        "top_p": 0.999,
        "stop_sequences": ["\n\nHuman:"]
    },
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

In [24]:
# We are also providing a different chat history retriever which outputs the history as a Claude chat (ie including the \n\n)
_ROLE_MAP = {"human": "\n\nUser: ", "ai": "\n\nAI: "}
def _get_chat_history(chat_history):
    buffer = ""
    for dialogue_turn in chat_history:
        if isinstance(dialogue_turn, BaseMessage):
            role_prefix = _ROLE_MAP.get(dialogue_turn.type, f"{dialogue_turn.type}: ")
            buffer += f"\n{role_prefix}{dialogue_turn.content}"
        elif isinstance(dialogue_turn, tuple):
            human = "\n\nUser: " + dialogue_turn[0]
            ai = "\n\nAI: " + dialogue_turn[1]
            buffer += "\n" + "\n".join([human, ai])
        else:
            raise ValueError(
                f"Unsupported chat history format: {type(dialogue_turn)}."
                f" Full chat history: {chat_history} "
            )
    return buffer

# 이전 대화 내용을 기반으로, 신규 질문을 재구성 하는 prompt
condense_prompt_claude = PromptTemplate.from_template("""
\n\nHuman:
Given the following conversation and a follow up question,
rephrase the follow up question to be a standalone question, in its original language.

Chat History: {chat_history}
Follow Up Input: {question}

\n\nAssistant: Question:"""
)

# memory
# store previous interactions using ConversationalBufferMemory and add custom prompts to the chat.
memory_chain = chat_utils.get_memory(
    memory_type="ConversationBufferMemory",
    memory_key="chat_history",
    return_messages=True
)

opensearch_hybrid_retriever.update_search_params(
    k=5,
    minimum_should_match=0,
    filter=[],
    reranker=True, # enable reranker with reranker model
    reranker_endpoint_name="huggingface-pytorch-inference-2023-11-15-04-37-45-120", # endpoint name for reranking model
    verbose=False
)

qa = ConversationalRetrievalChain.from_llm(
    llm=llm_text,
    retriever=opensearch_hybrid_retriever,
    memory=memory_chain,
    get_chat_history=_get_chat_history,
    verbose=True,
    condense_question_prompt=condense_prompt_claude,
    chain_type='stuff', # 'refine',
    #max_tokens_limit=300
)

# the LLMChain prompt to get the answer. the ConversationalRetrievalChange does not expose this parameter in the constructor
qa.combine_docs_chain.llm_chain.prompt = PromptTemplate.from_template("""
\n\nHuman:

{context}

<q></q> XML 태그 내의 질문에 답하려면 최대 3개의 문장을 사용하세요.

<q>{question}</q>

답변에 XML 태그를 사용하지 마세요.
답변이 context에 없으면 "죄송합니다. 문맥에서 답을 찾을 수 없어서 모르겠습니다."라고 말합니다.

\n\nAssistant:""")

채팅을 해보죠. 아래와 같은 질문을 해보세요. 
1. vefify DM

In [25]:
from utils.chat import ChatUX

In [26]:
chat = ChatUX(qa, retrievalChain=True)
chat.start_chat()

Starting chat bot


Output()

### Check momory

In [ ]:
#print_ww(memory.load_memory_variables({}))

### Reset memory

In [ ]:
# chat_utils.clear_memory(
#     chain=conversation
# )
# print_ww(memory.load_memory_variables({}))